In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count
from pyspark.sql.functions import pandas_udf

# df = spark.range(0, 10 * 1000 * 1000).withColumn('id', (col('id') / 1000).cast('integer')).withColumn('v', rand())
df = spark.range(0, 10 * 1000).withColumn('id', (col('id') / 1000).cast('integer')).withColumn('v1', rand()).withColumn('v2', rand())
df.cache()
df.count()

In [2]:
from pyspark.sql.functions import *
display(df.groupBy("id").agg(corr(col("v1"),col("v2"))))

In [3]:
import pandas as pd
from scipy import stats
group_column = 'id'
first_col = 'v1'
second_col = 'v2'
schema = df.select(group_column, first_col).schema

@pandas_udf(schema)
# Input/output are both a pandas.DataFrame
def group_corr(pdf):
    group_key = pdf[group_column].iloc[0]
    temp_corr = stats.pearsonr(pdf[first_col],pdf[second_col])[0]
    return pd.DataFrame([[group_key] + [temp_corr]], columns=[group_column] + ["corr_val"])
  
display(df.groupby(group_column).apply(group_corr))

In [4]:
df.groupby(group_column).apply(group_corr).explain()

In [5]:
df.groupBy("id").agg(corr(col("v1"),col("v2"))).explain()